In [1]:
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 60.7 MB/s eta 0:00:00


In [2]:
import mysql.connector
from mysql.connector import Error

hostname = "1wqn6.h.filess.io"
database = "olistproject_tankplenty"
port = "3307"
username = "olistproject_tankplenty"
password = "4b5c1fb653058ae4d33bfbefa084f931ab192bef"

try:
    connection = mysql.connector.connect(host=hostname, database=database, user=username, password=password, port=port)
    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)

except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")



<ipython-input-2-b1d12b1a72fb>:13: DeprecationWarning: Call to deprecated function get_server_info. Reason: 
    The property counterpart 'server_info' should be used instead.

  db_Info = connection.get_server_info()


Connected to MySQL Server version  8.0.36-28
You're connected to database:  ('olistproject_tankplenty',)
MySQL connection is closed


In [3]:
import pandas as pd
import mysql.connector
from mysql.connector import Error

# Connection details
hostname = "1wqn6.h.filess.io"
database = "olistproject_tankplenty"
port = "3307"
username = "olistproject_tankplenty"
password = "4b5c1fb653058ae4d33bfbefa084f931ab192bef"

# CSV file path
csv_file_path = "olist_order_payments_dataset.csv"
table_name = "olist_order_payments"

try:
  #Step 1: Establish a connection to MySQL server
    connection = mysql.connector.connect(
        host=hostname,
        database=database,
        user=username,
        password=password,
        port=port
    )
    if connection.is_connected():
        print("Connected to MySQL Server successfully!")

        cursor = connection.cursor()

        cursor.execute(f"DROP TABLE IF EXISTS {table_name};")
        print(f"Table `{table_name}` dropped if it existed.")

        create_table_query = f"""
        CREATE TABLE {table_name} (
            order_id VARCHAR(50),
            payment_sequential INT,
            payment_type VARCHAR(20),
            payment_installments INT,
            payment_value FLOAT
        );
        """
        cursor.execute(create_table_query)
        print(f"Table `{table_name}` created successfully!")

        data = pd.read_csv(csv_file_path)
        print("CSV data loaded into pandas DataFrame.")

        batch_size = 500
        total_records = len(data)

        print(f"Starting data insertion into `{table_name}` in batches of {batch_size} records.")
        for start in range(0, total_records, batch_size):
            end = start + batch_size
            batch = data.iloc[start:end]

            batch_records = [tuple(row) for row in batch.itertuples(index=False, name=None)]

            insert_query = f"""
            INSERT INTO {table_name}
            (order_id, payment_sequential, payment_type, payment_installments, payment_value)
            VALUES (%s, %s, %s, %s, %s);
            """
            cursor.executemany(insert_query, batch_records)
            connection.commit()
            print(f"Inserted records {start + 1} to {min(end, total_records)} successfully.")

        print(f"All {total_records} records inserted successfully into `{table_name}`.")

except Error as e:
    print("Error while connecting to MySQL or inserting data:", e)

finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed.")


Connected to MySQL Server successfully!
Table `olist_order_payments` dropped if it existed.
Table `olist_order_payments` created successfully!
CSV data loaded into pandas DataFrame.
Starting data insertion into `olist_order_payments` in batches of 500 records.
Inserted records 1 to 500 successfully.
Inserted records 501 to 1000 successfully.
Inserted records 1001 to 1500 successfully.
Inserted records 1501 to 2000 successfully.
Inserted records 2001 to 2500 successfully.
Inserted records 2501 to 3000 successfully.
Inserted records 3001 to 3500 successfully.
Inserted records 3501 to 4000 successfully.
Inserted records 4001 to 4500 successfully.
Inserted records 4501 to 5000 successfully.
Inserted records 5001 to 5500 successfully.
Inserted records 5501 to 6000 successfully.
Inserted records 6001 to 6500 successfully.
Inserted records 6501 to 7000 successfully.
Inserted records 7001 to 7500 successfully.
Inserted records 7501 to 8000 successfully.
Inserted records 8001 to 8500 successful

In [3]:
!pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 26.2 MB/s eta 0:00:00


In [4]:
# importing module
from pymongo import MongoClient
import pandas as pd

# Load the product_category CSV file into a pandas DataFrame
try:
    product_category_df = pd.read_csv("product_category_name_translation.csv")
except FileNotFoundError:
    print("Error: 'product_category_name_translation.csv' not found.")
    exit()  # Exit the script if the file is not found

# MongoDB connection details

hostname = "qv88g.h.filess.io"
database = "olistDataNoSQL_tenfivesay"
port = "61004"
username = "olistDataNoSQL_tenfivesay"
password = "b45f1511133dc6ced78982bd7284dfc6e2d713ef"

# Construct the MongoDB URI
uri = "mongodb://" + username + ":" + password + "@" + hostname + ":" + port + "/" + database

try:
    # Establish a connection to MongoDB
    client = MongoClient(uri)
    db = client[database]

    # Select the collection (or create if it doesn't exist)
    collection = db["product_categories"]  # Choose a suitable name for your collection

    # Convert the DataFrame to a list of dictionaries for insertion into MongoDB
    data_to_insert = product_category_df.to_dict(orient="records")

    # Insert the data into the collection
    collection.insert_many(data_to_insert)

    print("Data uploaded to MongoDB successfully!")

except Exception as e:
    # Handle any errors that occur during connection or insertion
    print(f"An error occurred: {e}")

finally:
    # Close the MongoDB connection
    if client:
        client.close()


Data uploaded to MongoDB successfully!
